# Demo: Serenity Factor Risk Model API

The heart of the Serenity API at this time are the risk functions. This notebook demonstrates how you can take a portfolio
assembled out of assets from our security master and compute a market risk attribution as of a given date. It also shows
how you can take that output and pivot the risk by asset, sector and factor to get a deeper understanding of the sources
of risk in your portfolio.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
%run -i init_demo.py

To run a risk attribution using the Serenity Factor Risk Model (SFRM) you first need to construct a portfolio. 
You can use a number of different symbologies, but for purposes of illustration we will use native blockchain symbol.

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')
positions = portfolio.to_asset_positions()

After this translation we have a portfolio using Serenity's internal identifiers:

In [ ]:
pd.json_normalize(positions)

The next thing we need is a model configuration. We can browse what's available with the Model API:

In [ ]:
model_meta = api.model().load_model_metadata(datetime.date.today())
configs = model_meta.get_model_configurations()
pd.DataFrame.from_dict(configs, orient='index', columns=['Description']).reset_index().rename(columns={'index': 'Short ID'})

Let's choose the medium-time horizon configuration for SFRM, currently the only one supported (and the default):

In [ ]:
from serenity_sdk.types import CalculationContext

# workaround until production upgraded
model_short_name = 'risk.factor.regression.SLM.MT'  
model_config_id = model_meta.get_model_configuration_id(model_short_name)

# construct the input parameters for risk attribution
ctx = CalculationContext(as_of_date='2021-07-01', model_config_id=model_config_id)

We can now run risk attribution. Past dates not yet cached may take longer to run (still typically less than 10 seconds):

In [ ]:
result = api.risk().compute_risk_attrib(ctx, portfolio)

The result object provides multiple Pandas DataFrames with different pivots. You can view the factor risks at a summary level:

In [ ]:
result.to_factor_risk_data_frame()

You can also break out risk by sectors, showing absolute and relative factor / specific / total risk at various levels,
allowing you to build a hierarchy:

In [ ]:
result.to_sector_risk_data_frame()

Or by sectors and factors:

In [ ]:
result.to_factor_sector_risk_data_frame()

Or by assets:

In [ ]:
result.to_asset_risk_data_frame(asset_master)